Fine-tuning best BART 🤖
-----------------------------------

In this notebook, we will continue the fine-tuning of the BART model on the new extracted sentences from the book **Grammaire de Wolof Moderne** and additional sentences. We provide, bellow, the main evaluation figures, obtained from the hyperparameter search step. We will evaluate the training on the validation dataset.

- Parallel coordinates from panel:

- Parameter importance char: 
[t5_v3_importance](https://wandb.ai/oumar-kane-team/small-t5-cross-fw-translation-bayes-hpsearch-v3/reports/undefined-23-05-16-10-36-17---Vmlldzo0Mzc4NDY0?accessToken=eyaiyrid0qz1zg2jkq3fc65biw53084dpfitbi0dgonq6mweupw6kgjml9d2nv1w)

We can see in the above chart that the batch is the most important parameter with a negative correlation with the BLEU score (meaning that a lower batch size is better). Next, we the probability of modifying a character in the french corpus is also important and a high probability provide a better BLEU score.  

In [1]:
# let us import all necessary libraries
from transformers import BartModel, BartForConditionalGeneration, Seq2SeqTrainer, BartTokenizerFast, set_seed, AdamW, get_linear_schedule_with_warmup,\
                          get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup, Adafactor
from wolof_translate.utils.sent_transformers import TransformerSequences
from wolof_translate.utils.improvements.end_marks import add_end_mark # added
from torch.nn import TransformerEncoderLayer, TransformerDecoderLayer
from torch.utils.data import Dataset, DataLoader, random_split
from wolof_translate.data.dataset_v4 import SentenceDataset # v2 -> v3 -> v4
from wolof_translate.utils.sent_corrections import *
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import _LRScheduler
# from custom_rnn.utils.kwargs import Kwargs
from torch.nn.utils.rnn import pad_sequence
from plotly.subplots import make_subplots
from nlpaug.augmenter import char as nac
from torch.utils.data import DataLoader
# from datasets  import load_metric # make pip install evaluate instead
# and pip install sacrebleu for instance
from torch.nn import functional as F
import plotly.graph_objects as go
from tokenizers import Tokenizer
import matplotlib.pyplot as plt
import pytorch_lightning as lt
from tqdm import tqdm, trange
from functools import partial
from torch.nn import utils
from copy import deepcopy
from torch import optim
from typing import *
from torch import nn
import pandas as pd
import numpy as np
import itertools
import evaluate
import random
import string
import shutil
import wandb
import torch
import json
import copy
import os

# add seed for everything
lt.seed_everything(0)

os.environ["WANDB_DISABLED"] = "true"

c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 0


## French to wolof

### Configure dataset 🔠

In [2]:
# recuperate the tokenizer from a json file
tokenizer = BartTokenizerFast(tokenizer_file=f"wolof-translate/wolof_translate/tokenizers/bart_tokenizers/tokenizer_v3_2.json")

In [3]:
def recuperate_datasets(fr_char_p: float, fr_word_p: float, max_len: int, end_mark_opt: int):

  # Let us recuperate the end_mark adding option
  if end_mark_opt == 1:
    # Create augmentation to add on French sentences
    fr_augmentation_1 = TransformerSequences(nac.KeyboardAug(aug_char_p=fr_char_p, aug_word_p=fr_word_p,
                                                             aug_word_max = max_len),
                                          remove_mark_space, delete_guillemet_space, add_mark_space)

    fr_augmentation_2 = TransformerSequences(remove_mark_space, delete_guillemet_space, add_mark_space)
    
  else:
    
    if end_mark_opt == 2:

      end_mark_fn = partial(add_end_mark, end_mark_to_remove = '!', replace = True)
    
    elif end_mark_opt == 3:

      end_mark_fn = partial(add_end_mark)
    
    elif end_mark_opt == 4:

      end_mark_fn = partial(add_end_mark, end_mark_to_remove = '!')

    # Create augmentation to add on French sentences
    fr_augmentation_1 = TransformerSequences(nac.KeyboardAug(aug_char_p=fr_char_p, aug_word_p=fr_word_p,
                                                             aug_word_max = max_len),
                                          remove_mark_space, delete_guillemet_space, add_mark_space, end_mark_fn)
    
    fr_augmentation_2 = TransformerSequences(remove_mark_space, delete_guillemet_space, add_mark_space, end_mark_fn)
    
  # Recuperate the train dataset
  train_dataset_aug = SentenceDataset(f"data/extractions/new_data/train_set.csv",
                                        tokenizer,
                                        truncation = True, max_len=max_len,
                                        cp1_transformer = fr_augmentation_1,
                                        cp2_transformer = fr_augmentation_2,
                                        add_bos_token=True
                                        )

  # Recuperate the valid dataset
  valid_dataset = SentenceDataset(f"data/extractions/new_data/valid_set.csv",
                                        tokenizer, max_len=max_len,
                                        cp1_transformer = fr_augmentation_2,
                                        cp2_transformer = fr_augmentation_2,
                                        add_bos_token=True,
                                        truncation = True)
  
  # Return the datasets
  return train_dataset_aug, valid_dataset

### Searching for the best parameters 🕖

In [4]:
from wolof_translate.models.transformers.optimization import TransformerScheduler
from wolof_translate.trainers.transformer_trainer import ModelRunner
from wolof_translate.utils.evaluation import TranslationEvaluation
from wolof_translate.models.transformers.main import Transformer
from wolof_translate.utils.split_with_valid import split_data


-------------

### --- Wandb v5 2000

In [12]:
# let us initialize the hyperparameter configuration 
config = {
    'random_state': 0,
    'fr_char_p': 0.19310415677880952,
    'fr_word_p': 0.07285606562901713,
    'learning_rate': 0.002762393179839311,
    'weight_decay': 0.033353528922291854,
    'batch_size': 16,
    'warmup_ratio': 0.0,
    'max_epoch': 454,
    'epochs': 45,
    'mid_epoch': 45,
    'max_len': 38,
    'end_mark': 3,
    'bleu': 7.3684,
    'model_dir': 'data/checkpoints/fw_bart_custom_train_v5_checkpoints',
    'new_model_dir': 'data/checkpoints/bart_custom_train_results_fw_v5'
}

# Initialize the model name
model_name = 'facebook/bart-base'

# import the model with its pre-trained weights
model = BartForConditionalGeneration.from_pretrained(model_name)

# resize the token embeddings
model.resize_token_embeddings(len(tokenizer))

# let us initialize the evaluation class
evaluation = TranslationEvaluation(tokenizer)

# let us initialize the trainer
trainer = ModelRunner(model, seed = 0, version = 5, evaluation = evaluation, optimizer=Adafactor)

# split the data
split_data(config['random_state'], csv_file = "ad_sentences.csv")

# recuperate train and test set
train_dataset, test_dataset = recuperate_datasets(config['fr_char_p'], 
                                                    config['fr_word_p'], config['max_len'],
                                                    config['end_mark'])

# let us calculate the appropriate warmup steps (let us take a max epoch of 100)
length = len(train_dataset)

n_steps = length // config['batch_size']

num_steps = config['max_epoch'] * n_steps

warmup_steps = (config['max_epoch'] * n_steps) * config['warmup_ratio']

# # Initialize the scheduler parameters
scheduler_args = {'num_warmup_steps': warmup_steps, 'num_training_steps': num_steps}

# Initialize the optimizer parameters
optimizer_args = {
    'lr': config['learning_rate'],
    'weight_decay': config['weight_decay'],
    # 'betas': (0.9, 0.98),
    'warmup_init': False,
    'relative_step': False
}

# Initialize the loaders parameters
train_loader_args = {'batch_size': config['batch_size']}

# Add the datasets and hyperparameters to trainer
trainer.compile(train_dataset, test_dataset, tokenizer, train_loader_args,
                optimizer_kwargs = optimizer_args,
                lr_scheduler=get_linear_schedule_with_warmup,
                lr_scheduler_kwargs=scheduler_args, 
                predict_with_generate = True,
                hugging_face = True,
                logging_dir="data/logs/bart_custom_train_fw"
                )

# We will from checkpoints so let us the model
# trainer.load(config['model_dir'], load_best=True) # Only for the first loading
trainer.load(config['new_model_dir'], load_best=True)

        

### --- Linear

In [11]:
trainer.train(epochs = config['epochs'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

  0%|          | 0/40 [00:00<?, ?it/s]c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\torch\optim\lr_scheduler.py:257: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


For epoch 6: 
{Learning rate: [0.0027708459507466156]}


Test batch number 13: 100%|██████████| 13/13 [00:13<00:00,  1.07s/batches]



Metrics: {'train_loss': 0.5331674415354777, 'test_loss': 1.0587910505441518, 'bleu': 6.9969, 'gen_len': 10.8283}




  2%|▎         | 1/40 [00:55<36:20, 55.90s/it]

...

 98%|█████████▊| 39/40 [42:33<01:01, 61.37s/it]

For epoch 45: 
{Learning rate: [0.00252768763959284]}


Test batch number 13: 100%|██████████| 13/13 [00:15<00:00,  1.21s/batches]



Metrics: {'train_loss': 0.018007289124370524, 'test_loss': 1.455772720850431, 'bleu': 15.0748, 'gen_len': 12.7172}




100%|██████████| 40/40 [43:33<00:00, 65.35s/it]


### --- Cosine

In [ ]:
trainer.train(epochs = config['epochs'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

### ---

In [ ]:
trainer.train(epochs = config['epochs'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

### ---

In [ ]:
trainer.train(epochs = config['epochs'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

### Predictions and Evaluation

In [13]:
# initialize the transformation sequence
end_mark_fn = partial(add_end_mark)
fr_augmentation = TransformerSequences(remove_mark_space, delete_guillemet_space, add_mark_space, end_mark_fn)

# let us get the test set
test_dataset = SentenceDataset(f"data/extractions/new_data/test_set.csv",
                                        tokenizer = tokenizer,
                                        cp1_transformer = fr_augmentation,
                                        cp2_transformer = fr_augmentation,
                                        truncation = True, add_bos_token=True)

Let us make the evaluation and print the predicted sentences.

In [14]:
# evaluation with test set
df_ft_to_wf = trainer.evaluate(test_dataset)

Evaluation batch number 13: 100%|██████████| 13/13 [00:44<00:00,  3.40s/batches]

predictions_: [[2, 0, 3684, 850, 136, 888, 5, 8, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [2, 0, 3673, 157, 1294, 13, 40, 282, 946, 82, 5, 8, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [2, 0, 3690, 850, 11, 5, 8, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [2, 0, 3684, 14, 14, 1175, 6, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [2, 0, 1574, 1625, 77, 2501, 23, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [2, 0, 3674, 17, 2029, 633, 6, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [2, 0, 3684, 14, 14, 1643, 274, 23, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [15]:
df_ft_to_wf[1].tail(10)

,original_sentences,translations,predictions
188,Donne le travail à un autre!,Joxal kenn liggéey bi!,Joxal téere bi doomu benn jigéen!
189,Cet homme qui a été.,Góor gii demoon.,Gor gii dem.
190,Dites-lui.,Nileen ka.,Nileen leen.
191,Il est là.,Ma ŋgoogule foofu.,Mi ŋgi fi.
192,Tu vois cet homme là-bas?,Gis ŋga nit kale?,Gis ŋga nit kee?
193,L'homme est parti je crois!,Ma defe góor gi dem na!,Góor gi dem na ma defe!
194,J'ai aperçu un baobab.,Séen naa ag guy.,Séen naa aw fas.
195,Celui-ci serait parti.,Kii dafa demkoon.,Kii dafa demkoon.
196,Toutes les portes étaient ouvertes.,Bunt yi yépp a tëjju woon.,Ci biir foofu yiooyale deey bëggu leen.
197,"C'est Fatim, aujourd'hui.","Faatim la, tay.",Faatim la soo demee.


In [16]:
# let us display 100 samples
pd.options.display.max_rows = 100
df_ft_to_wf[1].sample(100)

,original_sentences,translations,predictions
79,Nous fûmes Laobé.,Laobe lanu woon.,Laobe lañu woon.
48,"Quiconque s'en va, celui-là est un froussard.","Képp ku dem, kooku raggal la.","Kann, kookuleuñu dara."
68,Qu'ils ne partent pas!,Bu ñu dem!,Bu ñu dem!
38,Le gars est arrivé un peu en retard.,Góor gi egg na wanté tardé na tuuti.,Lépp jeex na.
60,Mes voeux d'anniversaire.,Mangui lay ndokkel ci sa bess bu délu si bi.,Mangui lay set.
40,Le voilà assis là-bas.,Ma ŋgii toog.,Ñu ŋgoogee.
100,Quel démon!,Moo di loola!,Moo di loola!
54,Ils ont tous convenu qu'ils travailleraient to...,Degganté ñu ñoom ñép ci lu ñép di liggéeyi ci ...,Laobe lañu woon.
52,"Et pourtant, je l'ai vu.","Moontin dey, gis naa ka.","Moontin, man, më dem."
127,C'est que j'ai effectivement été.,Dama dem.,Maa ŋgii demoon.


### Most similar tokens

In this subsection we want to find what tokens have most similarity between them using the embeddings. We must, at first, extract the weights of the embedding layer.

In [19]:
# the following can compare two vectors using the cosine similarity
def cosine_similarity(embed_1: torch.Tensor, embed_2: torch.Tensor):
  return np.dot(embed_1, embed_2)/(np.linalg.norm(embed_1) * np.linalg.norm(embed_2))

# recuperate the embeddings
embeddings = trainer.model.model.shared.weight

# let us compare the token at index 10 and 100
cosine_similarity(embeddings[10].cpu().detach().numpy(), embeddings[100].cpu().detach().numpy())

0.2508285

The cosine similarity is between 0 and 1. A higher value represents a higher similarity between two tokens.

To compare two groups of words we must tokenize them add the vectors of the tokens we obtain and calculate the similarity between the two groups' vectors.

Let us take the example of the words 'wax ak kii' and 'waxtaan ak yow'.

In [22]:
word1 = 'wax ak kii'

word2 = 'waxtaan ak yow'

# get the token ids
word1_id = tokenizer(word1, return_tensors='pt')['input_ids'][0]

word2_id = tokenizer(word2, return_tensors='pt')['input_ids'][0]

word1_id, word2_id


(tensor([2753,   73,  542]), tensor([2753,   31, 1116,   73,   48, 3452]))

Let us get their vectors with the following function.

In [37]:
def get_embedding(model: torch.nn.Module, word_id: torch.Tensor):
  
  embeddings = model.model.shared.weight.cpu().detach().numpy()
  
  return embeddings[word_id].mean(axis=0) if word_id.shape[0] else embeddings[word_id]

embeddings_1 = get_embedding(trainer.model, word1_id)

embeddings_2 = get_embedding(trainer.model, word2_id)

We can then calculate the cosine similarity between the embeddings.

In [39]:
similarity = cosine_similarity(embeddings_1, embeddings_2)

similarity

0.5306478

Let us add the two tasks inside an unique function.

In [45]:
def calculate_similary(model: torch.nn.Module, word1: str, word2: str):
  
  word1_id = tokenizer(word1, return_tensors='pt')['input_ids'][0]
  
  word2_id = tokenizer(word2, return_tensors='pt')['input_ids'][0]
  
  embeddings_1 = get_embedding(model, word1_id)
  
  embeddings_2 = get_embedding(model, word2_id)
  
  return cosine_similarity(embeddings_1, embeddings_2)

calculate_similary(trainer.model, 'wax ak kii', 'waxtaan ak yow')

0.5306478